<a href="https://colab.research.google.com/github/m-wessler/wr-stid-notebooks/blob/main/nbm/NBM4x_Percentile_In_Context.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#@markdown **RUN:** This first cell imports condaColab. <br><br>
#@markdown **NOTE: This cell will restart the notebook, which will prompt a crash popup in the lower left corner. This is safe to ignore and move on once the notebook comes back up** (you will see RAM and Disk in the upper right corner).

# !pip install -q condacolab
# import condacolab
# condacolab.install()

!apt-get -y install -qq libeccodes0
!pip -q install --no-deps shapely pyproj cartopy geopandas rasterio fiona xyzservices contextily eccodes cfgrib xarray netCDF4 cftime pygrib

import eccodes, cfgrib, xarray as xr, geopandas as gpd
print("eccodes module loaded:", hasattr(eccodes, 'codes_grib_new_from_file'))
print("cfgrib version:", cfgrib.__version__)
print("All imports successful!")

eccodes module loaded: True
cfgrib version: 0.9.15.0
All imports successful!


In [6]:
#@markdown **RUN:** This second cell imports our python modules

# Install required packages
# !mamba install -q -c conda-forge cartopy contextily pyepsg pygrib netCDF4

from IPython.display import Javascript

# General libraries
import numpy as np
import pandas as pd
import scipy.stats as stats

import json
import os
import re
import traceback
from datetime import datetime, timedelta
import zipfile
import itertools
import requests
from urllib.request import urlretrieve, urlopen

# Geospatial and projections
import pygrib
import geopandas as gpd
from pyproj import Proj, transform

# Plotting
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import seaborn as sns

# Cartopy and contextily for maps
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature

# Matplotlib configuration
matplotlib.rcParams['font.sans-serif'] = 'Liberation Sans'
matplotlib.rcParams['font.family'] = "sans-serif"

# Warnings
import warnings
warnings.filterwarnings("ignore")

In [8]:
#@title #<b>Options Selection { display-mode: "form" }
synoptic_token = "ee3af9f0107142f98764d24fbfff3348"  # @param {type:"string"}

element = "maxt" #@param ["maxt", "mint","qpf"]
valid_date = "2025-06-25" #@param {type:"date"}
#@markdown ##QPF valid 24 hours ending time
qpf_valid_time = 0 #@param {type:"slider", min:0, max:18, step:6}
##@markdown ##Use StageIV at stations instead of obs?
use_stageiv = False
##@param {type:"boolean"}
#@markdown ##Pick NBM run time
nbm_init_date = "2025-06-21" #@param {type:"date"}
nbm_init_hour = 0 #@param {type:"slider", min:0, max:18, step:6}
#@markdown ##Where do you want to focus?
# @markdown <FONT SIZE=5>**5. For Which Region?**
region_selection = "CWA" #@param ["WR", "SR", "CR", "ER", "CONUS", "CWA", "RFC"]
#@markdown If CWA/RFC selected, which one? (i.e. "SLC" for Salt Lake City, "CBRFC" for Colorado Basin)
cwa_selection = 'SLC' #@param {type:"string"}
# compare_to = "deterministic" #@param ["obs", "deterministic"]
#@markdown ##Which obs?
network_selection = "ALL" #@param ["NWS", "RAWS", "HADS", "SNOTEL", "NWS+RAWS", "NWS+RAWS+HADS", "ALL", "CUSTOM", "LIST"]
#@markdown ##Elevation selection (ft AMSL) <br>
#@markdown (Warning: Setting lower >= upper or upper <= lower will reset to default of 0-15000)
elev_lower_limit = 0 #@param {type:"slider", min:0, max:15000, step:250}
elev_upper_limit = 15000 #@param {type:"slider", min:0, max:15000, step:250}
#@markdown ##If Custom or List selected for network:
#@markdown Enter comma separated network IDs (custom) or siteids (list)  WITH NO SPACES here. For help - https://developers.synopticdata.com/about/station-providers/
network_input = ""#@param {type:"string"}
# cwa_outline = True #@param {type:"boolean"}
cwa_outline = False
#@markdown ##Do you want a CSV?
export_csv = False #@param {type:"boolean"}

# Setup a dictionary for translating a form selection into a something we can pass to mesowest API
network_dict = {"NWS+RAWS+HADS":"&network=1,2,106",
                "NWS+RAWS":"&network=1,2",
                "NWS":"&network=1",
                "RAWS": "&network=2",
                "HADS": "&network=106",
                "SNOTEL": "&network=25",
                "ALL":"",
                "CUSTOM": "&network="+network_input,
                "LIST": "&stid="+network_input}

network_string = network_dict[network_selection]

def cwa_list(input_region):
  region_dict ={"WR":"BYZ,BOI,LKN,EKA,FGZ,GGW,TFX,VEF,LOX,MFR,MTR,MSO,PDT,PSR,PIH,PQR,REV,STO,SLC,SGX,HNX,SEW,OTX,TWC",
              "CR":"ABR,BIS,CYS,LOT,DVN,BOU,DMX,DTX,DDC,DLH,FGF,GLD,GJT,GRR,GRB,GID,IND,JKL,EAX,ARX,ILX,LMK,MQT,MKX,MPX,LBF,APX,IWX,OAX,PAH,PUB,UNR,RIW,FSD,SGF,LSX,TOP,ICT",
              "ER":"ALY,LWX,BGM,BOX,BUF,BTV,CAR,CTP,RLX,CHS,ILN,CLE,CAE,GSP,MHX,OKX,PHI,PBZ,GYX,RAH,RNK,AKQ,ILM",
              "SR":"ABQ,AMA,FFC,EWX,BMX,BRO,CRP,EPZ,FWD,HGX,HUN,JAN,JAX,KEY,MRX,LCH,LZK,LUB,MLB,MEG,MFL,MOB,MAF,OHX,LIX,OUN,SJT,SHV,TAE,TBW,TSA"}
  if (input_region in ["WR", "CR", "SR", "ER"]):
    cwas_list = region_dict[input_region]
  else:
    cwas_list = input_region
  return cwas_list

def cwa_list_rfc(input_rfc):

    metadata_api = 'https://api.synopticdata.com/v2/stations/metadata?'

    network_query = (f"{network_dict[network_selection]}"
                    if network_dict[network_selection] is not None else '')

    # Assemble the API query
    api_query = (f"{metadata_api}&token={synoptic_token}" + network_query +
                f"&complete=1&sensorvars=1,obrange=20230118") #hardcoded for NBM4.1+

    # Print the API query to output
    print(api_query)

    # Get the data from the API
    response = requests.get(api_query)
    metadata = pd.DataFrame(response.json()['STATION'])

    # Remove NaNs and index by network, station ID
    metadata = metadata[metadata['MNET_SHORTNAME'].notna()]
    metadata = metadata.set_index(['MNET_SHORTNAME', 'STID'])

    metadata['LATITUDE'] = metadata['LATITUDE'].astype(float)
    metadata['LONGITUDE'] = metadata['LONGITUDE'].astype(float)
    metadata['ELEVATION'] = metadata['ELEVATION'].astype(float)

    metadata = metadata[metadata['LATITUDE'] >= 31]
    metadata = metadata[metadata['LONGITUDE'] <= -103.00]
    metadata = metadata[metadata['STATUS'] == 'ACTIVE']

    geometry = gpd.points_from_xy(metadata.LONGITUDE, metadata.LATITUDE)
    metadata = gpd.GeoDataFrame(metadata, geometry=geometry)

    req = requests.get(
        'https://www.weather.gov/source/gis/Shapefiles/WSOM/w_05mr24.zip',

    allow_redirects=True)
    open('w_05mr24.zip', 'wb').write(req.content)

    with zipfile.ZipFile('w_05mr24.zip', 'r') as zip_ref:
        zip_ref.extractall()

    rfc_shp = gpd.read_file('w_05mr24.shp').set_index('BASIN_ID')

    metadata = metadata[metadata.geometry.within(rfc_shp.geometry.loc[input_rfc])]

    rfc_site_list = metadata.index.get_level_values(1).unique()
    rfc_cwa_list = metadata['CWA'].unique()

    return metadata

if ((elev_lower_limit >= elev_upper_limit)
        or (elev_upper_limit <= elev_lower_limit)):
    elev_lower_limit = 0
    elev_upper_limit = 15000

if region_selection == "CONUS":
  region_list = ["WR", "CR", "SR", "ER"]
elif region_selection == "CWA":
  region_list = [cwa_selection]
elif region_selection == "RFC":
  rfc_metadata = cwa_list_rfc(cwa_selection)
#   cwa_query = ','.join([str(cwa) for cwa in rfc_metadata['CWA'].unique()
#     if cwa is not None])
  region_list = rfc_metadata['CWA'].unique()
  region_list = [x for x in region_list if x is not None]
else:
  region_list = [region_selection]

nbm_init = datetime.strptime(nbm_init_date,'%Y-%m-%d') + timedelta(hours=int(nbm_init_hour))

if element == "maxt":
    nbm_core_valid_hour="00"
    nbm_qmd_valid_hour="06"
    valid_date_start = datetime.strptime(valid_date,'%Y-%m-%d')
    valid_date_end = datetime.strptime(valid_date,'%Y-%m-%d') + timedelta(days=1)
    obs_start_hour = "1200"
    obs_end_hour = "0600"
    ob_stat = "maximum"
    valid_end_datetime = valid_date_end + timedelta(hours=(int(obs_end_hour)/100))
    nbm_core_valid_end_datetime = valid_date_end + timedelta(hours=int(nbm_core_valid_hour))
    nbm_qmd_valid_end_datetime = valid_date_end + timedelta(hours=int(nbm_qmd_valid_hour))
    core_init = nbm_init + timedelta(hours = 7)
    nbm_core_fhdelta = nbm_core_valid_end_datetime - core_init

elif element == "mint":
    nbm_core_valid_hour="12"
    nbm_qmd_valid_hour="18"
    valid_date_start = datetime.strptime(valid_date,'%Y-%m-%d')
    valid_date_end = datetime.strptime(valid_date,'%Y-%m-%d')
    obs_start_hour = "0000"
    obs_end_hour = "1800"
    ob_stat = "minimum"
    valid_end_datetime = valid_date_end + timedelta(hours=(int(obs_end_hour)/100))
    nbm_core_valid_end_datetime = valid_date_end + timedelta(hours=int(nbm_core_valid_hour))
    nbm_qmd_valid_end_datetime = valid_date_end + timedelta(hours=int(nbm_qmd_valid_hour))
    core_init = nbm_init + timedelta(hours = 7)
    nbm_core_fhdelta = nbm_core_valid_end_datetime - core_init

elif element == "qpf":
    nbm_core_valid_hour = str(qpf_valid_time)
    nbm_valid_hour = str(qpf_valid_time)
    nbm_qmd_valid_hour=str(qpf_valid_time)
    valid_date = datetime.strptime(valid_date,'%Y-%m-%d') + timedelta(hours=int(qpf_valid_time))
    valid_date_start = valid_date - timedelta(hours=24)
    valid_date_end = valid_date
    obs_start_hour = f'{qpf_valid_time:02d}00' #str(qpf_valid_time)+"00"
    obs_end_hour = f'{qpf_valid_time:02d}00' #str(qpf_valid_time)+"00"
    ob_stat = "total"
    valid_end_datetime = valid_date_end
    core_init = nbm_init
    nbm_core_valid_end_datetime = valid_date_end
    nbm_qmd_valid_end_datetime = valid_date_end
    nbm_core_fhdelta = nbm_core_valid_end_datetime - nbm_init

current_datetime = datetime.now()

nbm_core_forecasthour = nbm_core_fhdelta.total_seconds() / 3600.
nbm_core_forecasthour_start = nbm_core_forecasthour - 12
nbm_qmd_fhdelta = nbm_qmd_valid_end_datetime - nbm_init
nbm_qmd_forecasthour = nbm_qmd_fhdelta.total_seconds() / 3600.
if element == "qpf":
  nbm_qmd_forecasthour_start = nbm_qmd_forecasthour - 24
else:
  nbm_qmd_forecasthour_start = nbm_qmd_forecasthour - 18

statistics_api = "https://api.synopticlabs.org/v2/stations/legacystats?"
precipitation_api = "https://api.synopticdata.com/v2/stations/precipitation?"
metadata_api = "https://api.synopticdata.com/v2/stations/metadata?"

if element == "qpf":
  cmap = 'BrBG_r'
  txcol = 'white'
else:
  cmap = 'Spectral'
  txcol = 'black'
if use_stageiv and element=="qpf":
  points_str = f'Stage IV @ {network_selection}'
else:
  points_str = network_selection

########################################################################################################################
# Reusable functions section                                                                                           #
########################################################################################################################

def project3(lon, lat, prj):
  lon = float(lon)
  lat = float(lat)

  outproj = prj
  inproj = Proj(init='epsg:4326')
  nbm_coords = transform(inproj, outproj, lon, lat)
  coordX = nbm_coords[0]
  coordY = nbm_coords[1]
  #print(f'Lat: {lat}, Y: {coordY} | Lon: {lon}, X: {coordX}')
  return(coordX, coordY)


def ll_to_index(datalons, datalats, loclon, loclat):
  abslat = np.abs(datalats-loclat)
  abslon = np.abs(datalons-loclon)
  c = np.maximum(abslon, abslat)
  latlon_idx_flat = np.argmin(c)
  latlon_idx = np.unravel_index(latlon_idx_flat, datalons.shape)
  return(latlon_idx)


def project_hrap(lon, lat, s4x, s4y):
  lon = float(lon)
  lat = float(lat)

  globe = ccrs.Globe(semimajor_axis=6371200)
  hrap_ccrs = proj = ccrs.Stereographic(central_latitude=90.0,
                          central_longitude=255.0,
                          true_scale_latitude=60.0, globe=globe)
  latlon_ccrs = ccrs.PlateCarree()
  hrap_coords = hrap_ccrs.transform_point(lon,lat,src_crs=latlon_ccrs)
  hrap_idx = ll_to_index(s4x, s4y, hrap_coords[0], hrap_coords[1])

  return hrap_idx


def get_stageiv():
  siv_url = "https://water.weather.gov/precip/downloads/"+valid_date_end.strftime('%Y')+"/"+valid_date_end.strftime('%m')+"/"+valid_date_end.strftime('%d')+"/nws_precip_1day_"+valid_date_end.strftime('%Y%m%d')+"_conus.nc"
  data = urlopen(siv_url).read()

  nc = Dataset('data', memory=data)
  #with Dataset(siv_file, 'r') as nc:
  stageIV = nc.variables['observation']
  s4x = nc.variables['x']
  s4y = nc.variables['y']
  return stageIV, s4x, s4y


def K_to_F(kelvin):
  fahrenheit = 1.8*(kelvin-273)+32.
  return fahrenheit


def mm_to_in(millimeters):
  inches = millimeters * 0.0393701
  return inches


def find_roots(x,y):
  s = np.abs(np.diff(np.sign(y))).astype(bool)
  return x[:-1][s] + np.diff(x)[s]/(np.abs(y[1:][s]/y[:-1][s])+1)


# This bit of code to subset the grib and only download things we want is
# shamelessly stolen from Brian Blaylock (https://github.com/blaylockbk)
def download_subset(remote_url, remote_file, local_filename):
  print("   > Downloading a subset of NBM gribs")
  local_file = "nbm/"+local_filename
  if "qmd" in remote_file:
    if element == "maxt":
      if (int(nbm_qmd_forecasthour_start) % 24 == 0) and (int(nbm_qmd_forecasthour) % 24 ==0):
        search_string = f':TMP:2 m above ground:{str(int(int(nbm_qmd_forecasthour_start)/24))}-{str(int(int(nbm_qmd_forecasthour)/24))} day max fcst:'
      else:
        search_string = f':TMP:2 m above ground:{str(int(nbm_qmd_forecasthour_start))}-{str(int(nbm_qmd_forecasthour))} hour max fcst:'
    elif element == "mint":
      if (int(nbm_qmd_forecasthour_start) % 24 == 0) and (int(nbm_qmd_forecasthour) % 24 ==0):
        search_string = f':TMP:2 m above ground:{str(int(int(nbm_qmd_forecasthour_start)/24))}-{str(int(int(nbm_qmd_forecasthour)/24))} day min fcst:'
      else:
        search_string = f':TMP:2 m above ground:{str(int(nbm_qmd_forecasthour_start))}-{str(int(nbm_qmd_forecasthour))} hour min fcst:'
    elif element == "qpf":
      if (int(nbm_qmd_forecasthour_start) % 24 == 0) and (int(nbm_qmd_forecasthour) % 24 ==0):
        search_string = f':APCP:surface:{str(int(int(nbm_qmd_forecasthour_start)/24))}-{str(int(int(nbm_qmd_forecasthour)/24))} day acc fcst:'
      else:
        search_string = f':APCP:surface:{str(int(nbm_qmd_forecasthour_start))}-{str(int(nbm_qmd_forecasthour))} hour acc fcst:'
  elif "core" in remote_file:
    if element == "maxt":
      search_string = f':TMAX:2 m above ground:{str(int(nbm_core_forecasthour_start))}-{str(int(nbm_core_forecasthour))} hour max fcst:'
    elif element == "mint":
      search_string = f':TMIN:2 m above ground:{str(int(nbm_core_forecasthour_start))}-{str(int(nbm_core_forecasthour))} hour min fcst:'
  #print(search_string)
  idx = remote_url+".idx"
  r = requests.get(idx)
  if not r.ok:
    print('     ❌ SORRY! Status Code:', r.status_code, r.reason)
    print(f'      ❌ It does not look like the index file exists: {idx}')

  lines = r.text.split('\n')
  expr = re.compile(search_string)
  byte_ranges = {}
  for n, line in enumerate(lines, start=1):
      # n is the line number (starting from 1) so that when we call for
      # `lines[n]` it will give us the next line. (Clear as mud??)

      # Use the compiled regular expression to search the line
      if expr.search(line):
          # aka, if the line contains the string we are looking for...

          # Get the beginning byte in the line we found
          parts = line.split(':')
          rangestart = int(parts[1])

          # Get the beginning byte in the next line...
          if n+1 < len(lines):
              # ...if there is a next line
              parts = lines[n].split(':')
              rangeend = int(parts[1])
          else:
              # ...if there isn't a next line, then go to the end of the file.
              rangeend = ''

          # Store the byte-range string in our dictionary,
          # and keep the line information too so we can refer back to it.
          byte_ranges[f'{rangestart}-{rangeend}'] = line
          #print(line)
  for i, (byteRange, line) in enumerate(byte_ranges.items()):

        if i == 0:
            # If we are working on the first item, overwrite the existing file.
            curl = f'curl -s --range {byteRange} {remote_url} > {local_file}'
        else:
            # If we are working on not the first item, append the existing file.
            curl = f'curl -s --range {byteRange} {remote_url} >> {local_file}'
        try:
          num, byte, date, var, level, forecast, _ = line.split(':')
        except:
          pass

        #print(f'  Downloading GRIB line [{num:>3}]: variable={var}, level={level}, forecast={forecast}')
        os.system(curl)

  if os.path.exists(local_file):
      print(f'      ✅ Success! Searched for [{search_string}] and got [{len(byte_ranges)}] GRIB fields and saved as {local_file}')
      return local_file
  else:
      print(print(f'      ❌ Unsuccessful! Searched for [{search_string}] and did not find anything!'))



########################################################################################################################
# This section for downloading and processing obs                                                                      #
########################################################################################################################
print('Getting obs...')
obs = {}
for region in region_list:
  if (valid_end_datetime <= current_datetime):
    print("   > Grabbing obs for: ", region)
    #print("List of CWAs: ", cwa_list(region) )
    json_name = "obs/Obs_"+element+"_"+valid_date_start.strftime('%Y%m%d')+obs_start_hour+"_"+valid_date_end.strftime('%Y%m%d')+obs_end_hour+"_"+region+".json"
    if os.path.exists("obs"):
      pass
    else:
      !mkdir -p obs

    if element != "qpf":
      api_token = "&token="+synoptic_token
      station_query = "&cwa="+cwa_list(region)
      vars_query = "&vars=air_temp"
      start_query = "&start="+valid_date_start.strftime('%Y%m%d')+obs_start_hour
      end_query = "&end="+valid_date_end.strftime('%Y%m%d')+obs_end_hour
      stat_type = "&type="+ob_stat
      network_query = network_string
      api_extras = "&units=temp%7Cf&within=1440&status=active"
      obs_url = statistics_api + api_token + station_query + vars_query + start_query + end_query + stat_type + network_query + api_extras
      print(obs_url)
    elif element == "qpf":
      if use_stageiv:
        api_token = "&token="+synoptic_token
        station_query = "&cwa="+cwa_list(region)
        api_extras = "&fields=status,latitude,longitude,name,elevation"
        network_query = network_string
        obs_url = metadata_api + api_token + station_query + network_query + api_extras
        stageIV, s4xs, s4ys = get_stageiv()
        s4xs, s4ys = np.meshgrid(s4xs, s4ys)
      else:
        api_token = "&token="+synoptic_token
        station_query = "&cwa="+cwa_list(region)
        api_extras = "&fields=status,latitude,longitude,name,elevation&obtimezone=utc"
        network_query = network_string
        vars_query = "&pmode=totals"
        units_query = "&units=precip|in"
        start_query = "&start="+valid_date_start.strftime('%Y%m%d')+obs_start_hour
        end_query = "&end="+valid_date_end.strftime('%Y%m%d')+obs_end_hour
        obs_url = precipitation_api + api_token + station_query + network_query + vars_query + start_query + end_query + units_query + api_extras
      print(obs_url)
    if os.path.exists(json_name):
      pass
    else:
      urlretrieve(obs_url, json_name)
      print(obs_url)

    if os.path.exists(json_name):
        with open(json_name) as json_file:
            obs_json = json.load(json_file)
            obs_lats = []
            obs_lons = []
            obs_value = []
            obs_elev = []
            obs_stid = []
            obs_name = []
            for stn in obs_json["STATION"]:
                # print(stn.encode('utf-8'))
                if stn["STID"] is None:
                  stid = "N0N3"
                else:
                  stid = stn["STID"]
                #print(f'Processing {region} station {stid}')
                name = stn["NAME"]
                if stn["ELEVATION"] and stn["ELEVATION"] is not None:
                  elev = stn["ELEVATION"]
                else:
                  elev = -999
                lat = stn["LATITUDE"]
                lon = stn["LONGITUDE"]
                if element == "mint" or element=="maxt":
                  if 'air_temp_set_1' in stn['STATISTICS'] and stn['STATISTICS']['air_temp_set_1']:
                    if ob_stat in stn['STATISTICS']['air_temp_set_1'] and float(stn["LATITUDE"]) != 0. and float(stn["LONGITUDE"]) != 0.:
                      stat = stn['STATISTICS']['air_temp_set_1'][ob_stat]
                      obs_stid.append(str(stid))
                      obs_name.append(str(name))
                      obs_elev.append(float(elev))
                      obs_lats.append(float(lat))
                      obs_lons.append(float(lon))
                      obs_value.append(float(stat))
                elif (element == "qpf"):
                  if (stn["STATUS"] == "ACTIVE") and float(stn["LATITUDE"]) < 50.924 and float(stn["LATITUDE"]) > 23.377 and float(stn["LONGITUDE"]) > -125.650 and float(stn["LONGITUDE"]) < -66.008:
                    obs_stid.append(str(stid))
                    obs_name.append(str(name))
                    obs_elev.append(float(elev))
                    obs_lats.append(float(lat))
                    obs_lons.append(float(lon))
                    if use_stageiv:
                      coords = project_hrap(lon, lat, s4xs, s4ys)
                      siv_value = float(stageIV[coords])
                      if (siv_value >= 0.01):
                        obs_value.append(siv_value)
                      else:
                        obs_value.append(np.NaN)
                    else:
                      if "precipitation" in stn["OBSERVATIONS"]:
                        if "total" in stn["OBSERVATIONS"]["precipitation"][0]:
                          ptotal = stn["OBSERVATIONS"]["precipitation"][0]["total"]
                          if ptotal >= 0.01:
                            obs_value.append(ptotal)
                          else:
                            obs_value.append(np.nan)
                        else:
                          obs_value.append(np.nan)
                      else:
                        obs_value.append(np.nan)



            csv_name = "obs_"+element+"_"+region+".csv"
            obs[region] = pd.DataFrame()
            obs[region]["stid"] = obs_stid
            obs[region]["name"] = obs_name
            obs[region]["elevation"] = obs_elev
            obs[region]["lat"] = obs_lats
            obs[region]["lon"] = obs_lons
            obs[region]["ob_"+element] = obs_value
            #obs[region].to_csv(csv_name)

        # In order to make elevation subsetting work for multiple iterations
        os.remove(json_name)
  else:
    print(f'    > Valid Time in the future. Grabbing obs points only for: {region}')
    json_name = "obs/ObsPoints_"+region+".json"
    if os.path.exists(json_name):
      pass
    else:
      if os.path.exists("obs"):
        pass
      else:
        !mkdir -p obs
      obs_url = "https://api.synopticdata.com/v2/stations/metadata?&token="+synoptic_token+"&cwa="+cwa_list(region)+"&fields=status,latitude,longitude,name,elevation"+network_string
      urlretrieve(obs_url, json_name)
      print(obs_url)
    if os.path.exists(json_name):
      with open(json_name) as json_file:
          obs_json = json.load(json_file)
          obs_lats = []
          obs_lons = []
          obs_elev = []
          obs_stid = []
          obs_name = []
          for stn in obs_json["STATION"]:
            # print(stn.encode('utf-8'))
            if stn["STID"] is None:
              stid = "N0N3"
            else:
              stid = stn["STID"]
            #print(f'Processing {region} station {stid}')
            name = stn["NAME"]
            if stn["ELEVATION"] and stn["ELEVATION"] is not None:
              elev = stn["ELEVATION"]
            else:
              elev = -999
            lat = stn["LATITUDE"]
            lon = stn["LONGITUDE"]
            if stn["STATUS"] == "ACTIVE" and float(stn["LATITUDE"]) != 0. and float(stn["LONGITUDE"]) != 0.:
              obs_stid.append(str(stid))
              obs_name.append(str(name))
              obs_elev.append(float(elev))
              obs_lats.append(float(lat))
              obs_lons.append(float(lon))
          obs[region] = pd.DataFrame()
          obs[region]["stid"] = obs_stid
          obs[region]["name"] = obs_name
          obs[region]["elevation"] = obs_elev
          obs[region]["lat"] = obs_lats
          obs[region]["lon"] = obs_lons
          obs[region]["ob_"+element] = -999
          #obs[region].to_csv(csv_name)

    # In order to make elevation subsetting work for multiple iterations
    os.remove(json_name)

for region in region_list:
    querystr = f'{elev_lower_limit} <= elevation <= {elev_upper_limit}'
    print(f'Subsetting obs for region {region} by query: {querystr}')
    obs[region] = obs[region].query(querystr)

if region_selection == 'RFC':

  obs = pd.concat(obs).reset_index(
    ).rename(columns={"level_0":"cwa"}
    ).drop(columns=["level_1"]).set_index('stid')

  # Subset the indices to match
  obs_set = set(obs.index.unique())
  rfc_set = set(rfc_metadata.index.get_level_values(1).unique())
  rfc_site_index = np.array(list(obs_set.intersection(rfc_set)), dtype=str)
  obs = obs[obs.index.isin(rfc_site_index)]

  obs = {cwa_selection:obs}

  #Reset the region list
  region_list = [cwa_selection]

########################################################################################################################
# This section downloads and processes the NBM.                                                                        #
########################################################################################################################
print('Getting and processing NBM...')
nbm_init_filen = nbm_init.strftime('%Y%m%d') + "_" + nbm_init.strftime('%H')
nbm_init_filen_core = core_init.strftime('%Y%m%d') + "_" + core_init.strftime('%H')
nbm_url_base = "https://noaa-nbm-grib2-pds.s3.amazonaws.com/blend."+nbm_init.strftime('%Y%m%d') \
            +"/"+nbm_init.strftime('%H')+"/"
nbm_url_base_core = "https://noaa-nbm-grib2-pds.s3.amazonaws.com/blend."+core_init.strftime('%Y%m%d') \
            +"/"+core_init.strftime('%H')+"/"
temp_vars = ["maxt","mint"]
if (element == "qpf"):
  detr_file = f'blend.t{int(nbm_init_hour):02}z.qmd.f{int(nbm_qmd_forecasthour):03}.co.grib2'
  detr_file_subset = f'blend.t{int(nbm_init_hour):02}z.qmd.{nbm_init_filen}{nbm_init_filen}f{int(nbm_qmd_forecasthour):03}.co.{element}_subset.grib2'
  detr_url = nbm_url_base+"qmd/"+detr_file

elif any(te in element for te in temp_vars):
  detr_file = f"blend.t{int(core_init.strftime('%H')):02}z.core.f{int(nbm_core_forecasthour):03}.co.grib2"
  detr_file_subset = f"blend.t{int(core_init.strftime('%H')):02}z.core.{nbm_init_filen_core}f{int(nbm_core_forecasthour):03}.co.{element}_subset.grib2"
  detr_url = nbm_url_base_core+"core/"+detr_file


if os.path.exists("nbm/"+detr_file_subset):
  print("   > NBM deterministic already exists")
else:
  print("   > Getting NBM deterministic")
  if os.path.exists("nbm"):
    pass
  else:
    !mkdir -p nbm
  #urlretrieve(detr_url, "nbm/"+detr_file)
  download_subset(detr_url, detr_file, detr_file_subset)
#print(detr_url)
nbmd = pygrib.open("nbm/"+detr_file_subset)
if element == "maxt":
  try:
    deterministic = nbmd.select(name="Maximum temperature",lengthOfTimeRange=12, stepTypeInternal="max")[0]
  except:
    deterministic = nbmd.select(name="Time-maximum temperature",lengthOfTimeRange=12, stepTypeInternal="max")[0]
  deterministic_array = K_to_F(deterministic.values)
elif element == "mint":
  try:
    deterministic = nbmd.select(name="Minimum temperature",lengthOfTimeRange=12, stepTypeInternal="min")[0]
  except:
    deterministic = nbmd.select(name="Time-minimum temperature",lengthOfTimeRange=12, stepTypeInternal="min")[0]
  deterministic_array = K_to_F(deterministic.values)
elif element == "qpf":
  deterministic = nbmd.select(name="Total Precipitation",lengthOfTimeRange=24)[-1]
  deterministic_array = mm_to_in(deterministic.values)
nbmlats, nbmlons = deterministic.latlons()
nbmd.close()

for region in region_list:
  print("     >> Extracting NBM deterministic")
  point_lats = obs[region]["lat"].values
  point_lons = obs[region]["lon"].values
  detr_values = []
  nbm_fidx = []
  for i in range(0, len(point_lats)):
    coords = ll_to_index(nbmlons, nbmlats, point_lons[i], point_lats[i])
    detr_value = deterministic_array[coords]
    nbm_fidx.append(coords)
    detr_values.append(detr_value)
  obs[region]["NBM_fidx"] = nbm_fidx
  obs[region]["NBM_D"] = detr_values


perc_list = [1,5,10,20,30,40,50,60,70,80,90,95,99]
perc_list = np.array([i for i in range(1,100,1)])
perc_dict = {"maxt":"maxt18p", "mint":"mint18p", "qpf":"qpf24p"}
perc_file = f'blend.t{int(nbm_init_hour):02}z.qmd.f{int(nbm_qmd_forecasthour):03}.co.grib2'
perc_file_subset = f'blend.t{int(nbm_init_hour):02}z.qmd.{nbm_init_filen}{nbm_init_filen}f{int(nbm_qmd_forecasthour):03}.co.{element}_subset.grib2'
perc_url = nbm_url_base+"qmd/"+perc_file
if os.path.exists("nbm/"+perc_file_subset):
  print("   > NBM probabilistic already exists")
else:
  #urlretrieve(perc_url, "nbm/"+perc_file)
  print("   > Getting NBM probabilistic")
  download_subset(perc_url, perc_file, perc_file_subset)

nbmperc = pygrib.open("nbm/"+perc_file_subset)
print('   > Extracting NBM Probabilistic')
for perc in perc_list:
  print(f'     >> Extracting NBM P{int(perc):01}')
  perc_name = f"NBM_P{perc:02d}"#"NBM_P"+str(perc)

  if element == "maxt":
    try:
        percdata = K_to_F(nbmperc.select(name="Maximum temperature at 2 metres since previous post-processing", stepTypeInternal="max", percentileValue=perc)[0].values)
    except:
        percdata = K_to_F(nbmperc.select(name="Time-maximum 2 metre temperature", stepTypeInternal="max", percentileValue=perc)[0].values)
  elif element == "mint":
    try:
        percdata = K_to_F(nbmperc.select(name="Minimum temperature at 2 metres since previous post-processing", stepTypeInternal="min", percentileValue=perc)[0].values)
    except:
        percdata = K_to_F(nbmperc.select(name="Time-minimum 2 metre temperature", stepTypeInternal="min", percentileValue=perc)[0].values)
  elif element == "qpf":
    percdata = mm_to_in(nbmperc.select(name="Total Precipitation",lengthOfTimeRange=24, percentileValue=perc)[0].values)

  for region in region_list:
    nbm_coords = obs[region]["NBM_fidx"].values
    perc_values = []
    for i in range(0, len(nbm_coords)):
      perc_value = percdata[nbm_coords[i]]
      perc_values.append(perc_value)
    obs[region][perc_name] = perc_values
nbmperc.close()


########################################################################################################################
# This section creates a distribution curve at each site, and interpolates ob and deterministic to percentile space    #
# 3/27/2024 MW Removed spline interpolation in favor of explicit percentiles. Left spline and debug code if needed.    #
########################################################################################################################

print('Creating point distribution curves and interpolating...')
perc_list = np.array(perc_list)

for region in region_list:

  obs[region] = obs[region].sort_index(axis=1)

  perc_start = obs[region].columns.get_loc("NBM_P01")
  perc_end = obs[region].columns.get_loc("NBM_P99")
  all_percs = obs[region].iloc[:, perc_start:perc_end+1].values
  var_string = "ob_"+element
  all_obs = obs[region][[var_string]].values
  all_nbmd = obs[region][['NBM_D']].values
  obs_percs = []
  nbmd_percs = []

  for i in range(0,len(all_obs)):

    # udf = us(perc_list, all_percs[i,:], bbox=[0,100], ext=0)

    if all_obs[i] <= all_percs[i,:][0]:#udf(0):
      ob_perc = 0
    elif all_obs[i] >= all_percs[i,:][-1]:#udf(100):
      ob_perc = 100
    else:
    #   ob_perc = find_roots(np.arange(0,101,1), udf(np.arange(0,101,1)) - all_obs[i])
      ob_perc = find_roots(perc_list, all_percs[i,:] - all_obs[i])
      ob_perc = ob_perc[0].round(1)

    #   print('x: percentile list\n', np.arange(0,101,1), end='\n\n')
    #   print('udf(x)\n', udf(np.arange(0,101,1)), end='\n\n')
    #   print('all_obs[i]\n', all_obs[i], end='\n\n')
    #   print('y: udf(x) - ob value\n', udf(np.arange(0,101,1)) - all_obs[i], end='\n\n')

    # print('perc, perc_value, perc_value_interp')
    # for pp, pv in zip(perc_list, all_percs[i,:]):
    #     print(pp, pv, udf(pp))

    # print()
    # print('ob, ob_perc')
    # print(all_obs[i], ob_perc)

    if all_nbmd[i] <= all_percs[i,:][0]:#udf(0):
      nbm_perc = 0
    elif all_nbmd[i] >= all_percs[i,:][-1]:# udf(100):
      nbm_perc = 100
    else:
    #   nbm_perc = find_roots(np.arange(0,101,1), udf(np.arange(0,101,1)) - all_nbmd[i])
      nbm_perc = find_roots(perc_list, all_percs[i,:] - all_nbmd[i])
      nbm_perc = nbm_perc[0].round(1)

    # print('nbm_det, nbm_perc')
    # print(all_nbmd[i], nbm_perc)

    if np.isnan(ob_perc):
      obs_percs.append(ob_perc)
    else:
      obs_percs.append(int(ob_perc))

    nbmd_percs.append(int(nbm_perc))

    # print()
    # print('ranked against raw percentiles (coarse-step10)')
    perc_list=np.array(perc_list)
    # print(find_roots(perc_list, all_percs[i,:]-all_obs[i]))


  obs[region]["ob_perc"] = obs_percs
  obs[region]["NBMd_perc"] = nbmd_percs

  if export_csv:
    csv_name = "obs_"+element+"_"+valid_end_datetime.strftime('%Y%m%d')+"_"+region+".csv"
    obs[region].to_csv(csv_name)
    print(f'   > Created and saved {csv_name}')

########################################################################################################################
# Finally, this section makes our plot.                                                                                #
########################################################################################################################
def resize_colab_cell():
  display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'))

get_ipython().events.register('pre_run_cell', resize_colab_cell)
get_ipython().events.register('pre_run_cell', resize_colab_cell)

reg_cwa = region if region_selection != "CWA" else cwa_selection

if (valid_end_datetime <= current_datetime):
    compare_plots = ['obs', 'deterministic']
else:
    compare_plots = ['deterministic']

for compare_to in compare_plots:

    print("Making plot (almost done!)...")
    if compare_to =="obs":
        compare_var = "ob_perc"
        compare_element = "Obs"
    elif compare_to == "deterministic":
        compare_var = "NBMd_perc"

        if element =="qpf":
            compare_element = "Detr"
        else:
            compare_element = "Detr"

    title_dict = {"maxt":["Max T","PMaxT"],"mint":["Min T","PMinT"], "qpf":["QPF","PQPF"]}

    if (element == "qpf"):
        valid_datetime = valid_date
        fig_valid_date = valid_datetime.strftime('%Y%m%d_%HZ')
        valid_title = valid_datetime.strftime('%HZ %a %m-%d-%Y')
    else:
        valid_datetime = datetime.strptime(valid_date,'%Y-%m-%d')
        fig_valid_date = valid_datetime.strftime('%Y%m%d')
        valid_title = valid_datetime.strftime('%a %m-%d-%Y')
    nbm_init_title = nbm_init.strftime('%HZ %m-%d-%Y')

    if ((region_selection == "CWA") or (region_selection == "RFC")):
        dataframeid = cwa_selection
    else:
        dataframeid = region_selection

    mean = obs[dataframeid][compare_var].mean()
    median = obs[dataframeid][compare_var].median()
    mode = obs[dataframeid][compare_var].mode().values[0]

    # Create a subplot figure with 1 row and 2 columns
    fig = make_subplots(rows=1, cols=2,
                        column_widths=[0.5, 0.5],  # Adjust size ratio between map and histogram
                        specs=[[{"type": "xy"}, {"type": "scattermapbox"}]])

    # Histogram plot

    counts, bins = np.histogram(obs[dataframeid][compare_var], bins=range(0, 101, 10))
    bins = 0.5 * (bins[:-1] + bins[1:])
    hist_fig = px.bar(x=bins, y=counts, labels={'x':'Percentile Rank', 'y':'Counts'},
                    opacity=0.7) #, text_auto=True)
    fig.update_traces(marker_color='aqua')

    # Kernel Density Estimation (KDE)
    kde_x = np.linspace(0, 100, 1000)
    kde_y = stats.gaussian_kde(obs[dataframeid][compare_var].dropna())(kde_x)

    # Add KDE line to the same subplot
    fig.add_trace(
        go.Scatter(x=kde_x, y=kde_y * np.max(counts) / np.max(kde_y),
                mode='lines', name='KDE', showlegend=False,
                line=dict(color='aqua', width=2)),
        row=1, col=1)

    # Add histogram trace to subplot
    for trace in hist_fig.data:
        fig.add_trace(trace, row=1, col=1)

    # Add vertical lines for mean and median
    fig.add_trace(
        go.Scatter(x=[mean, mean], y=[0, np.max(counts)],
                mode='lines', name=f'Mean ({int(mean)})', line=dict(color='red', dash='dash', width=3)),
        row=1, col=1)

    fig.add_trace(
        go.Scatter(x=[median, median], y=[0, np.max(counts)],
                mode='lines', name=f'Median ({int(median)})', line=dict(color='green', width=3)),
        row=1, col=1)

    # Set the x-axis for the histogram on the left to always show 0 to 100
    fig.update_layout(
        xaxis=dict(range=[0, 100]),  # Set x-axis range from 0 to 100
        xaxis2=dict(range=[0, 100]), # Also apply it to the histogram subplot
        bargap=0,  # Remove space between bars
    )

    obs[dataframeid]['size'] = 30
    obs[dataframeid][f'{compare_var}_str'] = obs[dataframeid][compare_var].round(0).astype(str)
    #hacky fix for now
    obs[dataframeid][f'{compare_var}_str'] = [x.replace('.0', '') for x in obs[dataframeid][f'{compare_var}_str']]

    # Scatter mapbox plot with explicit range_color and coloraxis
    map_fig = px.scatter_mapbox(obs[dataframeid].dropna(), lat="lat", lon="lon",
                                color=compare_var, opacity=0.9,
                                size='size',
                                mapbox_style="open-street-map",
                                color_continuous_scale=cmap, range_color=[0, 100],  # Force color range from 0 to 100
                                text=f'{compare_var}_str',
                                hover_data={
                                    'name': True,  # Display the name of the point
                                    'stid': True,  # Display the station ID
                                    'elevation': True,
                                    f'{compare_var}': True,  # Display the observed percentile rank
                                    'NBMd_perc': True,  # Display the NBM percentile rank
                                    'NBM_D': True,
                                    f'{"ob_"+element}':True,
                                    f'{compare_var}_str': False,  # Display the observed percentile rank
                                    'size': False,  # Hide size from hover
                                    'lat': False,   # Hide latitude from hover
                                    'lon': False,   # Hide longitude from hover
                                })

    map_fig.update_traces(textposition='middle center', textfont=dict(color=txcol, weight='bold'))

    # Add scatter mapbox trace to subplot
    for trace in map_fig.data:
        fig.add_trace(trace, row=1, col=2)

    # Update layout with coloraxis and color range
    fig.update_layout(
        width=1600,
        height=800,
        bargap=0,
        mapbox=dict(
            center={"lat": obs[dataframeid]['lat'].mean(), "lon": obs[dataframeid]['lon'].mean()},
            zoom=6
        ),
        mapbox_style="open-street-map",
        coloraxis=dict(
            colorscale=cmap,
            colorbar=dict(
                title="<b>Percentile Rank</b>",
                tickfont=dict(size=18),
                tickvals=[0, 20, 40, 60, 80, 100],  # Ensure the colorbar ticks go from 0 to 100
                ticktext=["0", "20", "40", "60", "80", "100"],  # Matching labels
            ),
            cmin=0,  # Set minimum color scale value
            cmax=100,  # Set maximum color scale value
        ),
        margin={"r":0,"t":50,"l":0,"b":0},
        legend=dict(
            x=0.01,  # Position the legend to the left of the plot
            y=0.99,  # Align it to the top of the plot
            traceorder='normal',
            font=dict(size=18),
            bordercolor="Black",
            borderwidth=2,
        ),
        title_text=f'<b>{reg_cwa} {title_dict[element][0]} {compare_element} in NBM {title_dict[element][1]} Percentile Space</b> <br> Valid: {valid_title}  |  NBM Init: {nbm_init_title}  |  Points: {points_str}',
        title_x=0.5,  # title_y=0.98,
        title_font=dict(size=18),
        xaxis=dict(
            tickfont=dict(size=18)
        ),
        yaxis=dict(
            tickfont=dict(size=18)
        ),
    )

    fig.show(config={'scrollZoom':True})

Getting obs...
   > Grabbing obs for:  SLC
https://api.synopticlabs.org/v2/stations/legacystats?&token=ee3af9f0107142f98764d24fbfff3348&cwa=SLC&vars=air_temp&start=202506251200&end=202506260600&type=maximum&units=temp%7Cf&within=1440&status=active
https://api.synopticlabs.org/v2/stations/legacystats?&token=ee3af9f0107142f98764d24fbfff3348&cwa=SLC&vars=air_temp&start=202506251200&end=202506260600&type=maximum&units=temp%7Cf&within=1440&status=active
Subsetting obs for region SLC by query: 0 <= elevation <= 15000
Getting and processing NBM...
   > Getting NBM deterministic
   > Downloading a subset of NBM gribs
      ✅ Success! Searched for [:TMAX:2 m above ground:101-113 hour max fcst:] and got [2] GRIB fields and saved as nbm/blend.t07z.core.20250621_07f113.co.maxt_subset.grib2
     >> Extracting NBM deterministic
   > Getting NBM probabilistic
   > Downloading a subset of NBM gribs
      ✅ Success! Searched for [:TMP:2 m above ground:108-126 hour max fcst:] and got [108] GRIB fields a

Making plot (almost done!)...
